## Basic methods plus correlation pipeline with Feature-engine

In this notebook, we will apply basic methods to remove constant, quasi-constant and duplicated features, followed up by removing correlated features, in 1 single step, using Feature-engine and the Scikit-learn Pipeline.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from feature_engine.selection import (
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,)

In [2]:
data = pd.read_csv('../dataset_1.csv')
data.shape

(50000, 301)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape   # Separated dataset

((35000, 300), (15000, 300))

**Stack all the selection methods inside a pipeline!**

In [4]:
pipe = Pipeline([
    ('constant', DropConstantFeatures(tol=0.998)),
    ('duplicated', DropDuplicateFeatures()),
    ('correlation', SmartCorrelatedSelection(selection_method='variance')),])
pipe.fit(X_train)

Pipeline(steps=[('constant', DropConstantFeatures(tol=0.998)),
                ('duplicated', DropDuplicateFeatures()),
                ('correlation',
                 SmartCorrelatedSelection(selection_method='variance'))])

**Remove features!**

In [5]:
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)
X_train.shape, X_test.shape

((35000, 78), (15000, 78))

**The function to build logistic regression and compare performance in train and test set!**

In [6]:
def run_logistic(X_train, X_test, y_train, y_test):
    logit = LogisticRegression(random_state=44, max_iter=500)
    logit.fit(X_train, y_train)
    print('Train set')
    pred = logit.predict_proba(X_train)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = logit.predict_proba(X_test)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

**Filter methods - correlation!**

In [7]:
scaler = StandardScaler().fit(X_train)
run_logistic(scaler.transform(X_train),
             scaler.transform(X_test),
                  y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7919681799130189
Test set
Logistic Regression roc-auc: 0.7886120083307194
